In [ ]:
import os 
import glob
import numpy as np 
import pandas as pd

os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling/')
samplesheet_dir = 'results/samplesheets/chipseq/'
batch = 'batch1'

## Load the GSM Data

In [ ]:
fn = os.path.join(samplesheet_dir, 'ChIP-seq-Tracker-Batch1-2016-10.22.23-Step0.HiChIP-to-ChIP-seq-Map.tsv')
data = pd.read_table(fn, skiprows=1)
data = data.loc[data['Has ChIP-seq?'] == 'Yes']

In [ ]:
data = pd.read_table(fn, skiprows=1)

data = data.loc[data['Has ChIP-seq?'] == 'Yes']

In [ ]:
chipseq_gse_lists = data['GSM ID(s)'].dropna().tolist() + data['Control GSM(s)'].dropna().tolist()
chipseq_gsms = []
for gse_list in chipseq_gse_lists:
    gsm_ids = [x.strip() for x in gse_list.split(',')]
    chipseq_gsms.extend(gsm_ids)
    
chipseq_gsms = sorted(set(chipseq_gsms))

In [ ]:
data = data.loc[data['Has ChIP-seq?'] == 'Yes']

Query the below within the SRA Run Selector: https://www.ncbi.nlm.nih.gov/Traces/study/

In [ ]:
','.join(chipseq_gsms)

In [ ]:
len(chipseq_gsms)

## Parse the Queried Data

In [ ]:
# load the data
sra_table_fn = os.path.join(samplesheet_dir, 'chipseq.{}.SraRunTable.txt'.format(batch))
gsm_to_srr_data = pd.read_table(sra_table_fn, sep=',')
gsm_to_srr_data = gsm_to_srr_data[['Sample Name', 'Run']]
gsm_to_srr_data.columns = ['gsm_id', 'srr_id']
gsm_to_srr_data.sort_values('gsm_id', inplace=True)

In [ ]:
# # manually adding missing samples, these are from batch 2 not batch 1
# adding_missing_entries = [['GSM7336681', 'SRR16538542'],
#                           ['GSM7336682', 'SRR16538550'],
#                           ['GSM7336684', 'SRR16538541'],
#                           ['GSM7336685', 'SRR16538549'],
# ]
# adding_missing_entries = pd.DataFrame(adding_missing_entries, columns=gsm_to_srr_data.columns.tolist())
# gsm_to_srr_data = pd.concat([gsm_to_srr_data, adding_missing_entries], axis=0)

## Check for Discrepancies

In [ ]:
original_gses = set(chipseq_gsms)

In [ ]:
queried_gses = set(gsm_to_srr_data.gsm_id.unique())

In [ ]:
len(queried_gses)

In [ ]:
original_gses.difference(queried_gses)

Not sure how I got these extra stragglers 

In [ ]:
queried_gses.difference(original_gses)

In [ ]:
gsm_to_srr_data.gsm_id.value_counts()

## Create the Samplesheet for Downloading

In [ ]:
# get unique SRRs 

In [ ]:
srr_ids = sorted(gsm_to_srr_data.srr_id.unique())

In [ ]:
srr_ids_fn = os.path.join(samplesheet_dir, 'chipseq.srr_ids.{}.txt'.format(batch))
with open(srr_ids_fn, 'w') as fw:
    fw.write('\n'.join(srr_ids))

In [ ]:
srr_ids_fn

## Create the Samplesheet for Concatination

In [ ]:
# get Combinations of SRRs that should be concatinated
gsm_srr_combos_data = []

# main GSMs
for gsm_ids_combo in data['GSM ID(s)'].tolist():
    gsm_ids = gsm_ids_combo.replace(' ', '').split(',')
    gsm_combo_reps = [gsm_ids_combo] * len(gsm_ids)
    split_data = list(zip(gsm_combo_reps, gsm_ids))
    gsm_srr_combos_data.extend(split_data)    
    
# control GSMs
for gsm_ids_combo in data['Control GSM(s)'].tolist():
    
    if type(gsm_ids_combo) == str:
        gsm_ids = gsm_ids_combo.replace(' ', '').split(',')
        gsm_combo_reps = [gsm_ids_combo] * len(gsm_ids)
        split_data = list(zip(gsm_combo_reps, gsm_ids))
    else:
        split_data = [[np.nan, np.nan]]
    gsm_srr_combos_data.extend(split_data)    

In [ ]:
gsm_srr_combos_data = pd.DataFrame(gsm_srr_combos_data, columns=['GSM ID(s)', 'split_gsm_ids']).drop_duplicates()
gsm_srr_combos_data = gsm_srr_combos_data.merge(gsm_to_srr_data, left_on='split_gsm_ids', right_on='gsm_id', how='outer', indicator=True)
gsm_srr_combos_data = gsm_srr_combos_data.loc[~gsm_srr_combos_data.gsm_id.isna()]

In [ ]:
gsm_srr_combos_data_agg = gsm_srr_combos_data.groupby('GSM ID(s)').agg({'srr_id': '-'.join})
srr_combos = sorted(gsm_srr_combos_data_agg.srr_id.unique())

In [ ]:
srr_combos_fn = os.path.join(samplesheet_dir, 'chipseq.srr_id.combos.{}.txt'.format(batch))
with open(srr_combos_fn, 'w') as fw:
    fw.write('\n'.join(srr_combos))

### len(srr_combos)

## Create the Samplesheet for Alignment of Control SRRs

#### Generate a samplesheet that contains the organism

In [ ]:
# extracting gsms
df1 = data[['GSM ID(s)', 'Organism']].rename(columns={'GSM ID(s)': 'gsm_ids'})
df1['sample_type'] = 'main'
df2 = data[['Control GSM(s)', 'Organism']].rename(columns={'Control GSM(s)': 'gsm_ids'})
df2['sample_type'] = 'control'

# concating the gsm dfs
gsm_to_organism = pd.concat([df1, df2], axis=0).dropna().drop_duplicates()

# adding srr information
gsm_to_organism = gsm_to_organism.merge(gsm_srr_combos_data_agg, left_on='gsm_ids', right_index=True)

# extract control only
gsm_to_organism_control = gsm_to_organism.loc[gsm_to_organism.sample_type == 'control']

# adding reference genome information
human_subset = gsm_to_organism_control.loc[gsm_to_organism_control['Organism'] == 'Homo Sapiens']
mouse_subset = gsm_to_organism_control.loc[gsm_to_organism_control['Organism'] == 'Mus Musculus']

grch38_subset = human_subset.copy().sort_values(['srr_id'])
grch38_subset.loc[:, 'ref_genome'] = 'grch38'

t2t_subset = human_subset.copy().sort_values(['srr_id'])
t2t_subset.loc[:, 'ref_genome'] = 't2t'

mm10_subset = mouse_subset.copy().sort_values(['srr_id'])
mouse_subset.loc[:, 'ref_genome'] = 'mm10'

# concating all the subsets for a final samplesheet
gsm_to_organism_control = pd.concat([grch38_subset, mouse_subset, t2t_subset])

In [ ]:
# extract srr and ref_genome
gsm_to_organism_control = gsm_to_organism_control[['srr_id', 'ref_genome']]

In [ ]:
samplesheet_path = os.path.join(samplesheet_dir, 'chipseq.srr_ids_to_ref_genome.combos.{}.txt'.format(batch))
gsm_to_organism_control.to_csv(samplesheet_path, sep='\t', header=False, index=False)

## Create the Samplesheet for Running ChIPLine

In [ ]:
# process only uniqe GSM combos
chipline_ss = data[['Sample Name', 'GSE ID For ChIP-seq Data', 'Organism', 'Pulldown', 'GSM ID(s)', 'Control GSM(s)']]
chipline_ss = chipline_ss.drop_duplicates(subset=['GSM ID(s)'])

# add the srr information
chipline_ss = chipline_ss.merge(gsm_srr_combos_data_agg, left_on='GSM ID(s)', right_index=True, how='left')
chipline_ss.rename(columns={'srr_id': 'Main SRR(s)'}, inplace=True)
chipline_ss = chipline_ss.merge(gsm_srr_combos_data_agg, left_on='Control GSM(s)', right_index=True, how='left')
chipline_ss.rename(columns={'srr_id': 'Control SRR(s)'}, inplace=True)

# add organism infor
chipline_ss.loc[:, 'Organism'] = chipline_ss.loc[:, 'Organism'].str.replace('Homo Sapiens', 'Homo_Sapiens')
chipline_ss.loc[:, 'Organism'] = chipline_ss.loc[:, 'Organism'].str.replace('Mus Musculus', 'Mus_Musculus')

# adding the biorep information 
chipline_ss_tmp_list = []
sort_cols = ['GSE ID For ChIP-seq Data', 'Sample Name', 'Organism', 'Pulldown', 'GSM ID(s)', 'Control GSM(s)']
chipline_ss = chipline_ss.sort_values(sort_cols)
for grp, grp_df in chipline_ss.groupby(['GSE ID For ChIP-seq Data', 'Sample Name', 'Organism', 'Pulldown']):
    grp_df.loc[:, 'Rep No.'] = ['b{}'.format(x) for x in range(1, grp_df.shape[0] + 1)]
    chipline_ss_tmp_list.append(grp_df)
chipline_ss = pd.concat(chipline_ss_tmp_list)

In [ ]:
# add the standard sample name 
def create_std_sample_name(sr):
    name = '{}.{}.{}.{}.{}'.format(sr['Sample Name'], sr['GSE ID For ChIP-seq Data'],
                                   sr['Organism'], sr['Pulldown'], sr['Rep No.'])
    return(name)
chipline_ss.loc[:, 'chipseq_std_sample_name'] = chipline_ss.apply(create_std_sample_name, axis=1)

# organize the reference genome of samples
chipline_ss.loc[:, 'Ref Genome'] = chipline_ss['Organism'].map({'Homo_Sapiens': 'grch38',
                                                                 'Mus_Musculus': 'mm10'})
# add T2T samples
chipline_ss_grps = chipline_ss.groupby('Ref Genome')
t2t_chipline_ss = chipline_ss_grps.get_group('grch38').copy(deep=True)
t2t_chipline_ss.loc[:, 'Ref Genome'] = 't2t'

In [ ]:
# generate teh final chipline samplesheet
chipline_ss = pd.concat([chipline_ss, t2t_chipline_ss])

In [ ]:
# organize the columns 
chipline_ss = chipline_ss[['chipseq_std_sample_name', 'Main SRR(s)', 'Control SRR(s)', 'Ref Genome',
                           'Sample Name', 'GSE ID For ChIP-seq Data',
                            'Organism', 'Pulldown', 'Rep No.', 'GSM ID(s)', 'Control GSM(s)']]

In [ ]:
# finalize and save
chipline_ss.fillna('N/A', inplace=True)
chipline_ss_fn = os.path.join(samplesheet_dir, 'chipseq.chipline.{}.txt'.format(batch))
chipline_ss.to_csv(chipline_ss_fn, sep='\t', index=False, header=False)

## Add the ChIP-seq file that finally maps between HiChIP and ChIP-seq

In [ ]:
mapper = data.copy()
mapper.rename(columns={'Replicate Serial No': 'Rep No.'}, inplace=True)
mapper.loc[:, 'Organism'] = mapper.loc[:, 'Organism'].str.replace(' ' , '_') 
mapper.loc[:, 'Rep No.'] = ['b{}'.format(x) for x in mapper.loc[:, 'Rep No.'].astype(int)]
mapper.loc[:, 'chipseq_std_sample_name'] = mapper.apply(create_std_sample_name, axis=1)

def get_chipseq_path(std_sample_name):
    
    res_dir = 'results/peaks/chipline_v2/'
    sample_tmpl = os.path.join(res_dir, '{0}/MACS2_Ext_*/{0}.macs2_peaks.narrowPeak_Q0.01filt'.format(std_sample_name))
    print(sample_tmpl)
    peak_fns = glob.glob(sample_tmpl)
    
    if len(peak_fns) == 1:
        return(peak_fns[0])
    elif len(peak_fns) > 1:
        return('Found multiple files')
    else:
        return('Could not find a matching file')

mapper.loc[:, 'chipseq_path'] = mapper.loc[:, 'chipseq_std_sample_name'].apply(get_chipseq_path)
fn = os.path.join(samplesheet_dir, 'chipseq.tracker.hichip_to_chipseq.paths_mapped.{}.tsv'.format(batch))
mapper.to_csv(fn, sep='\t', index=False, header=True)

In [ ]:
mapper